<a href="https://colab.research.google.com/github/manosai2005/prodigy-ML-4/blob/main/prodigy_ml_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d gti-upm/leapgestrecog

Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
100% 2.12G/2.13G [00:19<00:00, 81.7MB/s]
100% 2.13G/2.13G [00:19<00:00, 119MB/s] 


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/leapgestrecog.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
import os
import pandas as pd

def gather_image_paths_and_labels(root_dir):
    file_paths = []
    labels = []

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_paths.append(os.path.join(root, file))
                label = os.path.basename(os.path.dirname(root))
                labels.append(label)

    return file_paths, labels


data_dir = '/content/leapGestRecog'

file_paths, labels = gather_image_paths_and_labels(data_dir)

df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

df.head()

,file_path,label
0,/content/leapGestRecog/00/01_palm/frame_00_01_...,00
1,/content/leapGestRecog/00/01_palm/frame_00_01_...,00
2,/content/leapGestRecog/00/01_palm/frame_00_01_...,00
3,/content/leapGestRecog/00/01_palm/frame_00_01_...,00
4,/content/leapGestRecog/00/01_palm/frame_00_01_...,00


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 224, 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_generator = datagen.flow_from_dataframe(
    df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 16000 validated image filenames belonging to 10 classes.
Found 4000 validated image filenames belonging to 10 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = len(train_generator.class_indices)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
500/500 [==============================] - 76s 136ms/step - loss: 0.1612 - accuracy: 0.9454 - val_loss: 18.9976 - val_accuracy: 0.0000e+00
Epoch 2/10
500/500 [==============================] - 71s 143ms/step - loss: 0.0255 - accuracy: 0.9888 - val_loss: 21.4902 - val_accuracy: 0.0000e+00
Epoch 3/10
500/500 [==============================] - 68s 137ms/step - loss: 0.0184 - accuracy: 0.9902 - val_loss: 24.3766 - val_accuracy: 0.0000e+00
Epoch 4/10
500/500 [==============================] - 68s 136ms/step - loss: 0.0175 - accuracy: 0.9896 - val_loss: 27.6189 - val_accuracy: 0.0000e+00
Epoch 5/10
500/500 [==============================] - 68s 136ms/step - loss: 0.0216 - accuracy: 0.9892 - val_loss: 21.2287 - val_accuracy: 0.0000e+00
Epoch 6/10
500/500 [==============================] - 72s 144ms/step - loss: 0.0170 - accuracy: 0.9904 - val_loss: 18.3309 - val_accuracy: 0.0000e+00
Epoch 7/10
500/500 [==============================] - 69s 139ms/step - loss: 0.0153 - accuracy: 0.99

In [ ]:
validation_loss, validation_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {validation_acc}")

125/125 [==============================] - 13s 102ms/step - loss: 33.7037 - accuracy: 0.0000e+00
Validation Accuracy: 0.0


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_hand_gesture(img_path, model):
    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])

    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    gesture = class_indices[predicted_class]

    return gesture


img_path = '/content/frame_00_01_0001.png'
gesture = predict_hand_gesture(img_path, model)
print(f"Predicted Gesture: {gesture}")

1/1 [==============================] - 1s 611ms/step
Predicted Gesture: 06
